# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from IPython.display import display
import string                          #library of string constants
import re
from sqlalchemy import create_engine
import nltk                            # natural language tool kit
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import pickle
import os
os.path.abspath(os.getcwd())
os.chdir('C:\\Users\\Jim.000\\Documents\\Udacity Data Scientist Class\\Pipeline\\sturges_disaster_response_pipeline_project/')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jim.000\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jim.000\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jim.000\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('messages', engine)
X = df['message']
Y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
cols=list(Y.columns)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    Description: This function reads a text string
                    removes punctuation
                    converts to lower case
                    strips out white space
                    tokenizes the text into a word list
                    Stems the word list
                    removes stop words

    Arguments: Initial test string
    Returns: clean list of word tokens
    """
    stop_words = nltk.corpus.stopwords.words("english")
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())
    #tokenize
    words = nltk.word_tokenize (text)
    #stem
    #A processing interface for removing morphological affixes from words. This process is known as stemming.
    # (https://www.nltk.org/api/nltk.stem.html)
    clean_tokens = [nltk.PorterStemmer().stem(w) for w in words if w not in stop_words]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
forest_clf = RandomForestClassifier(n_estimators=10)
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(forest_clf))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.33, random_state = 42)

pipeline.fit(X_train.values, Y_train.values)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def report_results(y_test, y_pred):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
    from sklearn.metrics import precision_recall_fscore_support
    
    """
    Description: This function reads in the Y test and the Y predicted values
                    it used the sklearn precision_recall_fscore_support function
                    to compute the fscore, precision and recall values for each catigory
                    it places them in a dataframe for nice formating
                    lastly it compute the average of these score to provide and view
    Arguments: the y test and y predicted values
    Returns: a summary report
    """
    report = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])
    for indx, cat in enumerate(y_test.columns):
        idx=indx+1
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,indx], average='weighted')
        report.at[idx,'Category']=cat
        report.at[idx,'f_score']=f_score
        report.at[idx,'precision']=precision
        report.at[idx,'recall']=recall

    rd={}
    rd['Precision (averaged)']=report['precision'].mean()
    rd['Recall (averaged)']=report['recall'].mean()
    rd['F_score (averaged)']=report['f_score'].mean()
    print(pd.DataFrame.from_dict(rd,orient='index').rename(columns={0:"Score"}))
    report.set_index('Category', inplace=True)
    return report

print(report_results(Y_test, y_pred))


#from sklearn.metrics import classification_report
#for i, cat in enumerate(Y_test):
    #print(cat)
    #print(classification_report(Y_test[cat], y_pred[:, i]))
  


                         Score
Precision (averaged)  0.933867
Recall (averaged)     0.944300
F_score (averaged)    0.932141
                         f_score precision    recall
Category                                            
related                  0.78657  0.784583  0.798544
request                 0.871651  0.874966  0.883957
offer                   0.993417  0.991235  0.995608
aid_related             0.744015  0.747884  0.749191
medical_help            0.891011  0.894612   0.92025
medical_products        0.933649  0.942193  0.951687
search_and_rescue       0.962489  0.966504  0.974226
security                0.972289  0.963354  0.981392
military                0.959859   0.95953  0.969834
child_alone                    1         1         1
water                    0.94281  0.946423  0.952381
food                    0.928717  0.929751  0.934697
shelter                 0.918279  0.923656  0.931577
clothing                0.984183  0.985899  0.987748
money                   0.96

C:\Users\Jim.000\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
def improve_model(pipeline,X_train,Y_train):
    """
    Description: This function reads in a previously built pipeline from the sklearn library
    (https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
    then a gridsearch (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
    using diferent parameter (currently hardcoded in this model but they could be passed in)
    
    Inputs:
    pipeline: predefined using sklearn
    X: iterable Training data
    y: iterable Training data

    Arguments: the y test and y predicted values
    Returns: fitted, tunned model
    """
   
    from sklearn.model_selection import GridSearchCV
    parameters = {'vect__min_df': [5],
                  'tfidf__use_idf':[True, False],
                  'clf__estimator__n_estimators':[25], 
                  'clf__estimator__min_samples_split':[2, 5]}

    #scorer = make_scorer(performance_metric)
    cv = GridSearchCV(pipeline, param_grid = parameters,  verbose = 1)


    # Find best parameters
    np.random.seed(81)
    tuned_model = cv.fit(X_train, Y_train)
    tuned_model=tuned_model.best_estimator_
    return tuned_model

tuned_model=improve_model(pipeline,X_train,Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 39.7min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
y_pred = tuned_model.predict(X_test)

print(report_results(Y_test, y_pred))







C:\Users\Jim.000\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                         Score
Precision (averaged)  0.939062
Recall (averaged)     0.948519
F_score (averaged)    0.938392
                         f_score precision    recall
Category                                            
related                 0.792924  0.794397  0.808252
request                 0.886183  0.890864  0.896556
offer                   0.993417  0.991235  0.995608
aid_related             0.766733  0.766975  0.766528
medical_help            0.901658  0.903834   0.92337
medical_products        0.938682  0.945279  0.953652
search_and_rescue       0.964702   0.96689  0.974688
security                0.972174   0.96335   0.98116
military                0.960392  0.959438  0.969718
child_alone                    1         1         1
water                   0.958929  0.960618  0.963477
food                    0.947414  0.947153  0.949723
shelter                 0.932049  0.934268  0.940129
clothing                0.983026  0.984459  0.987055
money                   0.97

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [13]:
pickle.dump(tuned_model,open('models/classifier.pkl','wb'))



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.